# Crypto Cheat Sheet — Jupyter Notebook Friendly

Kratke "implementacijske specifikacije" (kaj naj funkcija stori, vhod/izhodi, kratek test/skica). Namen: hitro prepisovanje v Jupyter celice.

---

# 1) Zamična šifra (Caesar)

**Funkcija:** `caesar_encrypt(text: str, shift: int) -> str` / `caesar_decrypt(text, shift)`

* **Vhod:** ASCII/letters, `shift` 0..25
* **Izhod:** premaknjeno besedilo, ohranja velikost črk in ne-črke
* **Test:** `decrypt(encrypt("ABC",3),3) == "ABC"`
* **Skica:** `C = (P + s) mod 26`

```python
def caesar_encrypt(text, shift):
    # premakne črke za shift
    pass
```

---

# 2) Zamenjalna šifra (monoalphabetic substitution)

**Funkcija:** `sub_encrypt(text: str, mapping: dict) -> str`

* **Vhod:** `mapping` dict: `{'A':'Q', 'B':'W', ...}`
* **Izhod:** šifrirano besedilo
* **Test:** `decrypt(encrypt(x,m), inverse(m)) == x`
* **Skica:** vsaka črka preslikana po tabeli

```python
def sub_encrypt(text, mapping):
    # map each letter via mapping
    pass
```

---

# 3) Vigenère

**Funkcija:** `vigenere_encrypt(plaintext: str, key: str) -> str`

* **Vhod:** `key` (A-Z), ponavljamo ključ pri ne-črkah pa ohranimo
* **Izhod:** šifrat
* **Test:** decrypt(encrypt(m,k),k)==m
* **Skica:** `Ci = (Pi + Ki) mod 26`

```python
def vigenere_encrypt(pt,key):
    pass
```

---

# 4) Enojna podloga (One-Time Pad)

**Funkcija:** `otp_encrypt(plaintext: bytes, key: bytes) -> bytes`

* **Vhod:** `len(key) >= len(plaintext)`, key must be random
* **Izhod:** ciphertext = bytes(a ^ b)
* **Test:** `xor(ct,key) == pt`
* **Skica:** `C = P XOR K` ; *ne uporabljaj K večkrat*

```python
def otp_encrypt(pt,key):
    assert len(key) >= len(pt)
    return bytes(a^b for a,b in zip(pt,key))
```

---

# 5) Tokovna šifra (general)

**Funkcija:** `stream_encrypt(plaintext: bytes, keystream: bytes) -> bytes`

* **Vhod:** keystream generated by PRG (same length)
* **Izhod:** ciphertext = XOR
* **Test:** reuse keystream -> vulnerability (show C1^C2)

```python
def stream_encrypt(pt,ks):
    return bytes(a^b for a,b in zip(pt,ks))
```

---

# 6) LCG (Linear Congruential Generator)

**Funkcija:** `lcg(seed:int,a:int,c:int,m:int,n:int) -> bytes`

* **Vhod:** params (m large), n bytes
* **Izhod:** pseudorandom bytes (state % 256 each step)
* **Test:** deterministic for same seed
* **Opomba:** *not cryptographically secure*

```python
def lcg(seed,a,c,m,n):
    s = seed
    out = bytearray()
    for _ in range(n):
        s = (a*s + c) % m
        out.append(s % 256)
    return bytes(out)
```

---

# 7) Dva potencialno varna PRG-ja

**A) HMAC-DRBG / HMAC-expand (HKDF-like)**

* **Funkcija:** `prg_hmac(seed: bytes, n: int) -> bytes`
* **Opis:** iterativni HMAC(seed, last || counter)
* **Test:** isti seed -> isti keystream; različni seed -> različni

**B) AES-CTR as PRG**

* **Funkcija:** `prg_aes_ctr(key: bytes, iv: bytes, n: int) -> bytes`
* **Opis:** encrypt counter blocks under AES key
* **Test:** deterministic; use cryptographic library in practice

```python
# HMAC-PRG sketch
import hmac,hashlib

def prg_hmac(seed,n):
    out=bytearray(); last=b''; ctr=1
    while len(out)<n:
        mac=hmac.new(seed, last+bytes([ctr]), hashlib.sha256).digest()
        out.extend(mac); last=mac; ctr+=1
    return bytes(out[:n])
```

---

# 8) ECB (Electronic Codebook)

**Funkcija:** `ecb_encrypt(blocks: List[bytes], key) -> List[bytes]`

* **Vhod:** blokovna šifra (npr. AES blocksize)
* **Izhod:** vsak blok šifriran ločeno
* **Slabost:** enaki plaine -> enaki ciphers (pattern leak)

```python
def ecb_encrypt(blocks, encrypt_block):
    return [encrypt_block(b) for b in blocks]
```

---

# 9) CBC (Cipher Block Chaining)

**Funkcija:** `cbc_encrypt(plaintext: bytes, key, iv: bytes, encrypt_block)`

* **Vhod:** pad plaintext to block size
* **Izhod:** chaining: `C0 = IV`, `Ci = E(Pi XOR C{i-1})`
* **Test:** decrypt recovers plaintext; IV must be random

```python
def cbc_encrypt(blocks, iv, encrypt_block):
    prev = iv; out=[]
    for b in blocks:
        x = bytes(a^b for a,b in zip(b,prev))
        c = encrypt_block(x)
        out.append(c); prev=c
    return b"".join(out)
```

---

#10) CTR (Counter mode)
**Funkcija:** `ctr_encrypt(plaintext: bytes, key, nonce, encrypt_block)`

* **Opis:** keystream = E(nonce || ctr) for ctr=0..; XOR with plaintext
* **Izhod:** parallelizable, random nonce required

```python
def ctr_keystream(encrypt_block, nonce, length):
    out=bytearray(); ctr=0
    while len(out)<length:
        block = encrypt_block(nonce + ctr.to_bytes(8,'big'))
        out.extend(block); ctr+=1
    return bytes(out[:length])
```

---

#11) MAC (Message Authentication Code)
**Funkcija:** `mac(key: bytes, message: bytes) -> bytes`

* **Opis:** generic: produce fixed-length tag, e.g. HMAC
* **Test:** compare with stored tag using constant-time compare

---

#12) CBC-MAC
**Funkcija:** `cbc_mac(key, message, encrypt_block)`

* **Opis:** CBC encryption with zero IV; output = last block
* **Opozorilo:** insecure for variable-length messages unless fixed-length or use CMAC

```python
def cbc_mac(blocks, encrypt_block):
    prev = b"\x00"*BLOCK
    for b in blocks:
        prev = encrypt_block(bytes(x^y for x,y in zip(prev,b)))
    return prev
```

---

#13) HMAC
**Funkcija:** `hmac_sha256(key,message)`

* **Opis:** HMAC(K,M) = H((K^opad)||H((K^ipad)||M))
* **Test:** use `hmac.compare_digest` for verification

---

#14) CMAC
**Funkcija:** `cmac_aes(key,message)`

* **Opis:** AES-based MAC (recommended over CBC-MAC), use library or implement subkey generation
* **Test:** verify against known test vectors

---

#15) ECBC-MAC (Error: maybe Means encrypted CBC-MAC?)
**Opomba:** verjetno mislil *EMAC* ali *Encrypted CBC-MAC*; glavna ideja: CBC-MAC + encrypt last block.

* **Funkcija:** `ecbc_mac = E_k(CBC_MAC)`
* **Opomba:** raba v nekaterih starostnih protokolih

---

#16) Carter-Wegman (Universal Hash + one-time pad)
**Funkcija:** `cw_mac(key_hash, key_enc, message) -> tag`

* **Opis:** compute universal hash (polynomial) over message, then encrypt with one-time pad or XOR with key to get tag
* **Test:** different key_hash -> different tags; used in Wegman-Carter MAC constructions

---

#17) Hash functions (general)
**Funkcija:** `hash(data: bytes) -> bytes`

* **Opis:** preimage resistance, second-preimage, collision resistance
* **Test:** small change -> large change (avalanche)

---

#18) Merkle–Damgård construction

* **Opis:** build hash from compression function `f(h_prev, block)`; pad message, iterate, output final state
* **Varnost:** collision resistance of hash <= collision resistance of compression + padding rules

---

#19) SHACAL-2
**Opomba:** SHACAL-2 is block cipher built from SHA-256; rarely implemented by hand in exam; function stub:

* **Funkcija:** `shacal2_encrypt_block(key, block)`
* **Opis:** convert SHA-256 rounds into encryption rounds

---

#20) AES-CBC
**Funkcija:** `aes_cbc_encrypt(pt, key, iv)`

* **Opis:** use AES block encrypt in CBC chaining; pad with PKCS#7
* **Test:** decrypt(encrypt(pt))==pt; use strong IV
* **Sketch code:** use `Crypto.Cipher.AES` in practice

---

#21) AES-CTR
**Funkcija:** `aes_ctr_encrypt(pt, key, nonce)`

* **Opis:** use AES to generate keystream from nonce||counter and XOR with pt
* **Test:** parallelizable and random-access

---

#22) GCM (Galois/Counter Mode)
**Funkcija:** `aes_gcm_encrypt(pt, key, iv, aad)`

* **Opis:** provides confidentiality + integrity (AEAD). Produces (ciphertext, tag)
* **Test:** use library; verify that modifying ciphertext or AAD fails verification

---

#23) CCM (Counter with CBC-MAC)
**Funkcija:** `aes_ccm_encrypt(pt, key, nonce, aad)`

* **Opis:** AEAD combining CBC-MAC and CTR; careful ordering and formatting of lengths
* **Test:** use library implementations

---

# Quick examples to paste into Jupyter

Below are tiny code snippets you can copy into notebook cells.

```python
# XOR helper
def xor_bytes(a,b):
    return bytes(x^y for x,y in zip(a,b))

# Caesar example
def caesar_encrypt(text,shift):
    out=[]
    for ch in text:
        if 'a'<=ch<='z':
            out.append(chr((ord(ch)-97+shift)%26 +97))
        elif 'A'<=ch<='Z':
            out.append(chr((ord(ch)-65+shift)%26 +65))
        else:
            out.append(ch)
    return ''.join(out)

# HMAC example
import hmac,hashlib

def hmac_sha256(key,msg):
    return hmac.new(key,msg,hashlib.sha256).digest()
```

---

# Notes & exam tips

* For implementations: always state block size, input assumptions, padding rules.
* When using MACs: always verify with constant-time compare.
* For PRG: mention seed uniqueness and non-reuse, prefer cryptographic PRG (HMAC/AES-CTR).
* For modes: ECB leaks patterns; prefer CBC with random IV, or CTR/GCM for AEAD.
* For OTP: never reuse key; key length ≥ message length.

---

Good luck! Shrani ta dokument kot Jupyter Markdown ("Markdown" celli) in dodaj kode v "Code" celli. Če želiš, naredim iz tega dejanski `.ipynb` file (export) ali Java različico.
